### Install Requirements

In [1]:
!pip install -r ./requirements.txt -q

### Create Tools

In [2]:
from keys import apikey, aws_access_key_id, aws_secret_access_key
import os
from langchain.tools import BaseTool
import boto3
import json

os.environ['OPENAI_API_KEY'] = apikey
os.environ['aws_access_key_id'] = aws_access_key_id
os.environ['aws_secret_access_key'] = aws_secret_access_key


class LambdaRMQ(BaseTool):
    name = 'Lambda function for RMQ firewall rules'
    description = """Do NOT use this tool until you have received all input arguments from the user directly.
    
                    Use this tool to invoke a lambda function to create security group rules for only "RMQ" or "RabbitMQ".
                    
                    If the port is not for "RabbitMQ" or "RMQ", do not create a rule. Respond that you cannot open that port.
                    
                    Do not attempt to create a security group rule that is not for "RabbitMQ" or "RMQ" with this tool.
                    
                    ONLY accept action_input or input of "RMQ" or "RabbitMQ".
        
                    DO NOT accept action_input or input of anything else.
                    """

    def _run(self, get_log=False):

        """
        Do NOT use this tool until you have asked for the 'cidr' input argument.
        
        Use this tool to invoke a lambda function to create security group rules for only "RMQ" or "RabbitMQ"
        
        If the security group rule is not for "RabbitMQ" or "RMQ", do not create a rule. Respond that you cannot create that rule. 

        Only invoke the lambda function for input of "RMQ" or "RabbitMQ".
        
        ONLY accept action_input or input of "RMQ" or "RabbitMQ".
        
        DO NOT accept action_input or input of anything else. 

        :return: The response from the function invocation.
        """
        
        client = boto3.client('lambda', region_name='us-east-1', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)

        cidr_ranges = {'vdi': '10.10.64.0/24', 'surface': '10.10.65.0/24', 'development': '192.168.0.0/24'}
        cidr = ''
        if cidr == '':
            try:
                cidr = cidr_ranges.get(input(str('Please provide the group that will need access. (for example; vdi, surface, or development)')).lower())
            except KeyError:
                print('That is not a valid entry')
                _run()
        else:
            return 'Something went wrong. Please try again.'
   
        
        inputParams = {
        'cidr'    : cidr
        }
        
        response = client.invoke(
            FunctionName=''# add name of function,
            Payload = json.dumps(inputParams)
            )
        return "I have created your security group rules for RMQ!"

    def _arun(self, sops):
        raise NotImplementedError("This tool does not support async")


class LambdaHTTPS(BaseTool):
    name = 'Lambda function for HTPPS firewall rules'
    description = """Use this tool to invoke a lambda function to create security group rules for only HTTPS.
                    
                    If the port is not for HTTPS, do not create a rule. Respond that you cannot open that port.
                    
                    Do not attempt to create a security group rule that is not for HTTPS with this tool.
                    
                    ONLY accept action_input or input of HTTPS.
        
                    DO NOT accept action_input or input of anything else.
                    
                    """

    def _run(self, function_name, get_log=False):

        """
        Use this tool to invoke a lambda function to create security group rules for only HTTPS
        
        If the port is not for HTTPS, do not create a rule. Respond that you cannot open that port.

        Only invoke the lambda function for input of HTTPS.
        
        ONLY accept action_input or input of HTTPS.
        
        DO NOT accept action_input or input of anything else.
        
        :return: The response from the function invocation.
        """
        
        client = boto3.client('lambda', region_name='us-east-1', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)

        response = client.invoke(
            FunctionName='' # add name of function
            )
        return "I have created your security group rules for HTTPS!"

    def _arun(self, sops):
        raise NotImplementedError("This tool does not support async")

### Create LLM, Memory, Agent, and Import Tools

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

llm = ChatOpenAI(
    openai_api_key=apikey,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=2,
    return_messages=True
)

In [4]:
from langchain.agents import initialize_agent

tools = [LambdaRMQ(), LambdaHTTPS()]

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

### Interact with the Agent

In [5]:
agent('Can you create security group rules for RabbitMQ?')



> Entering new AgentExecutor chain...
{
    "action": "Lambda function for RMQ firewall rules",
    "action_input": "RMQ"
}

Please provide the group that will need access. (for example; vdi, surface, or development) surface



Observation: I have created your security group rules for RMQ!
Thought:{
    "action": "Final Answer",
    "action_input": "The security group rules for RabbitMQ have been successfully created."
}

> Finished chain.


{'input': 'Can you create security group rules for RabbitMQ?',
 'chat_history': [],
 'output': 'The security group rules for RabbitMQ have been successfully created.'}

In [6]:
agent('Can you create a firewall rule for RDP?')



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "You cannot create a firewall rule for RDP using the provided tool. Please use a different method to create the firewall rule for RDP."
}

> Finished chain.


{'input': 'Can you create a firewall rule for RDP?',
 'chat_history': [HumanMessage(content='Can you create security group rules for RabbitMQ?', additional_kwargs={}),
  AIMessage(content='The security group rules for RabbitMQ have been successfully created.', additional_kwargs={})],
 'output': 'You cannot create a firewall rule for RDP using the provided tool. Please use a different method to create the firewall rule for RDP.'}